### Import Packages


In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes

In [137]:
training_dataroot = 'processed_data_weekday.csv'
testing_dataroot = 'processed_data_weekday_test.csv'
output_dataroot = 'lab1_basic.csv' 

training_datalist =  [] 
# testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] 

In [136]:
import pandas as pd

# 假設你的數據存儲在一個CSV文件中
df = pd.read_csv(testing_dataroot)

# 轉換 upload time 為 datetime 類型
df['upload time'] = pd.to_datetime(df['upload time'])

# 提取星期幾
df['day_of_week'] = df['upload time'].dt.dayofweek

# 進行 One-Hot 編碼
df = pd.get_dummies(df, columns=['day_of_week'], prefix='day')

# 計算 hash tag 數量
df['hashtag_count'] = df['hashtag'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)

# 將處理後的數據保存到新的CSV文件
df.to_csv('processed_data_weekday_test.csv', index=False)

print(df)


      Post Index  followers         upload time  Like Count  Comment Count  \
0              0    1539000 2024-11-18 09:42:12    336000.0            276   
1              1    4647000 2024-11-22 08:34:18    769000.0            114   
2              2    1113000 2024-11-23 14:50:50     85000.0            152   
3              3     221000 2024-11-22 11:38:19     31000.0            183   
4              4    9111000 2024-11-22 10:02:55    435000.0            157   
...          ...        ...                 ...         ...            ...   
3177         963      60000 2024-11-25 00:28:51         2.0              2   
3178         964     173000 2024-10-04 13:10:35     15000.0             29   
3179         965      28000 2024-11-08 15:29:04      6447.0             28   
3180         966      40000 2024-11-14 08:47:15     10000.0              3   
3181         967     159000 2024-11-13 10:32:49      2385.0              6   

      is Photo  is Video                                       

### Load the Input File

In [118]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot)
  training_datalist=training_datalist[training_datalist['is Photo'] == 1]

# with open(testing_dataroot, newline='') as csvfile:
#   testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [5]:
print(training_datalist[:,7])

["polarbear ,icecream ,aiart ,artcommunity ,cub ,cutebear ,art ,polar ,arctic ,ice-cream'"
 "Crispy fried chicken chop ,Crispy fried shrimp ,Hualien ,Hualien City ,Hualien Food ,Hualien Simple ,Hualien lunch ,Hualien Dinner ,Eat Hualien with great heart ,Picnic picnic ,Pine garden annex ,gourmet food ,Ingredients'"
 "Cause ,Stalk ,Oblinter stalk ,Mag ,memes ,Sharing ,joke ,Sneer ,funny ,Funny movie ,Humorous quotation ,change ,Lu Guangzhong ,singer ,muscle ,fitness'"
 ... "Soda ,sodagreen ,Soda ,sodagreen ,20thAnniversaryConcertTour'"
 "Black and white chef '" 'no hashtage']


### Implement the Regression Model

In [119]:
import numpy as np

# 假設 x_train 是你的數據
# 這裡以第一列為例
data = training_datalist[["followers", "is Photo" ,"Like Count"]].to_numpy()[:, 0]

# 計算第一四分位數和第三四分位數
Q1 = np.percentile(data, 25)
Q3 = np.percentile(data, 75)
IQR = Q3 - Q1

# 定義異常值範圍
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 篩掉異常值
filtered_indices = np.where((data >= lower_bound) & (data <= upper_bound))
filtered_data = training_datalist.to_numpy()[filtered_indices]




### Process Hashtag

In [120]:
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from bs4 import BeautifulSoup

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)

    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]
count = CountVectorizer(ngram_range=(1, 1),
                        preprocessor=preprocessor,
                        tokenizer=tokenizer_stem_nostop)

count.fit(training_datalist["hashtag"].to_numpy())
BoW = count.vocabulary_
print('[vocabulary]\n{}'.format(BoW))
doc_bag = count.transform(training_datalist["hashtag"].to_numpy())
doc_bag = doc_bag.toarray()
print(doc_bag)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sammy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\sammy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[vocabulary]
{'caus': 697, 'stalk': 4045, 'oblint': 3165, 'mag': 2577, 'meme': 2724, 'share': 3827, 'joke': 2129, 'sneer': 3942, 'funni': 1491, 'movi': 2929, 'humor': 1878, 'quotat': 3512, 'chang': 721, 'lu': 2546, 'guangzhong': 1673, 'singer': 3898, 'muscl': 2945, 'fit': 1412, 'hashtag': 1772, 'lowiq01': 2542, 'lowiq01andtherhythmmakersplu': 2543, 'challeng': 713, 'hibiya': 1812, 'outdoor': 3232, 'music': 2947, 'hall': 1727, 'ichikawa': 1908, 'entertain': 1261, 'bikost': 429, 'mannahero': 2640, 'tabl': 4185, 'bkmn': 445, 'bikosmanna': 428, 'educ': 1211, 'thisisvanlif': 4295, 'vanlifeproject': 4493, 'vanlif': 4486, 'vanlifestyl': 4495, 'vanlifediari': 4488, 'vanconvers': 4484, 'vanlifemov': 4492, 'sprintervanconvers': 4034, 'lifecom': 2431, 'sprintervan': 4033, 'sprinterconvers': 4032, 'sprintervanlif': 4035, 'vanbuild': 4482, 'vanlifeeurop': 4489, 'campervan': 636, 'enfurgomolam': 1248, 'houseonwheel': 1860, 'kitchenidea': 2240, 'greenkitchen': 1649, 'tinyhous': 4323, 'smallkitchen': 

In [121]:
def preProcessed(x, y):
    x = x.astype(np.float64) 
    y = y.astype(np.float64)
    y = y[:, np.newaxis]
    combined = np.hstack((x, y))
    nan_removed = combined[~np.isnan(combined).any(axis=1)]
    x_train = nan_removed[:, :-1]
    y_train = nan_removed[:, -1:]
    #scaler = StandardScaler()
    #x_train = scaler.fit_transform(x_train)
    #y_train = scaler.fit_transform(y_train)
    x_train[:, 0] = np.where(x_train[:, 0] > 0, np.log10(x_train[:, 0].clip(min=1e-10)), x_train[:, 0])
    #x_train[:, 2] = np.where(x_train[:, 2] > 0, np.log10(x_train[:, 2].clip(min=1e-10)), x_train[:, 2])
    #x_train[:, 3] = np.where(x_train[:, 3] > 0, np.log10(x_train[:, 3].clip(min=1e-10)), x_train[:, 3])
    y_train[:, 0] = np.where(y_train[:, 0] > 0, np.log10(y_train[:, 0].clip(min=1e-10)), y_train[:, 0])
    # x_train = (x_train - np.mean(x_train, axis=0)) / np.std(x_train, axis=0)
    # y_train = (y_train - np.mean(y_train)) / np.std(y_train)
    return x_train, y_train
def split_data(X_train, Y_train, split_ratio=0.2):

    x_split_size = int(len(X_train) * split_ratio)
    y_split_size = int(len(Y_train) * split_ratio)
    x_train = X_train[x_split_size:]
    y_train = Y_train[y_split_size:]
    x_val = X_train[:x_split_size]
    y_val = Y_train[:y_split_size]

    return x_train, y_train, x_val, y_val

In [122]:
xy = training_datalist[["followers", "is Photo" ,"hashtag_count","Like Count"]].to_numpy()
weekday=training_datalist[["day_0", "day_1","day_2","day_3","day_4","day_5","day_6"]].to_numpy()
print(xy)
x = xy[:, :-1]
x= np.hstack((x,weekday))  
x= np.hstack((x,doc_bag))
y = xy[:, -1]   
print(y)
x_processed, y_processed = preProcessed(x, y)
print(y_processed)
x_train, y_train, x_val, y_val = split_data(x_processed, y_processed, split_ratio=0.2)
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")


[[ 12000.      1.     16.   6817.]
 [102000.      1.      1.   4905.]
 [ 12000.      1.     16.   6887.]
 ...
 [   313.      1.      1.      1.]
 [    25.      1.      1.      2.]
 [     9.      1.      1.      3.]]
[6817. 4905. 6887. ...    1.    2.    3.]
[[3.8336]
 [3.6906]
 [3.838 ]
 ...
 [0.    ]
 [0.301 ]
 [0.4771]]
x_train shape: (2960, 4868), y_train shape: (2960, 1)
x_val shape: (740, 4868), y_val shape: (740, 1)


In [127]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# 假设数据已经准备好，x_train, y_train, x_val, y_val 是 numpy 数组

x_train[:, 1] = x_train[:, 1].astype(int)


# 初始化 CatBoost 回归模型
model = CatBoostRegressor(
    iterations=2000,             # 迭代次数
    learning_rate=0.001,          # 学习率
    depth=12,                    # 决策树深度
    loss_function='RMSE',       # 损失函数，回归任务用 RMSE
    random_seed=42,             # 随机种子，确保可复现性
    verbose=100                 # 每隔多少次输出训练信息
)

# 将数据转换为合适的格式
y_train = y_train.ravel()  # 确保 y_train 是一维数组
y_val = y_val.ravel()

cat_features = [1] 
model.fit(
    x_train, y_train,
    eval_set=(x_val, y_val),
    use_best_model=True
)

# 验证模型
y_pred = model.predict(x_val)

# 计算均方误差 (MSE)
mse = mean_squared_error(y_val, y_pred)
print(f"Validation MSE: {mse:.4f}")


0:	learn: 1.4309844	test: 1.3297493	best: 1.3297493 (0)	total: 137ms	remaining: 4m 33s
100:	learn: 1.3716628	test: 1.2588385	best: 1.2588385 (100)	total: 21.4s	remaining: 6m 41s
200:	learn: 1.3188244	test: 1.1950191	best: 1.1950191 (200)	total: 46.5s	remaining: 6m 56s
300:	learn: 1.2732009	test: 1.1387936	best: 1.1387936 (300)	total: 1m 11s	remaining: 6m 42s
400:	learn: 1.2331704	test: 1.0890506	best: 1.0890506 (400)	total: 1m 35s	remaining: 6m 22s
500:	learn: 1.1983131	test: 1.0459668	best: 1.0459668 (500)	total: 1m 59s	remaining: 5m 57s
600:	learn: 1.1674153	test: 1.0086480	best: 1.0086480 (600)	total: 2m 23s	remaining: 5m 34s
700:	learn: 1.1399112	test: 0.9760425	best: 0.9760425 (700)	total: 2m 47s	remaining: 5m 10s
800:	learn: 1.1162363	test: 0.9476272	best: 0.9476272 (800)	total: 3m 12s	remaining: 4m 48s
900:	learn: 1.0947564	test: 0.9225096	best: 0.9225096 (900)	total: 3m 38s	remaining: 4m 26s
1000:	learn: 1.0759166	test: 0.9005518	best: 0.9005518 (1000)	total: 4m 4s	remaining: 4

In [ ]:
def MAPE(y_true, y_pred):
    """
    Calculates Mean Absolute Percentage Error (MAPE) between the true and predicted values.

    Parameters:
    - y_true (numpy.ndarray): Actual values (ground truth).
    - y_pred (numpy.ndarray): Predicted values.

    Returns:
    - float: MAPE value as a percentage.
    """

    non_zero_indices = y_true != 0
    y_true_filtered = y_true[non_zero_indices]
    y_pred_filtered = y_pred[non_zero_indices]

    ape = np.mean(np.abs((y_true_filtered - y_pred_filtered) / y_true_filtered)) * 100

    return ape

mape = MAPE(y_val, y_pred)
print(f"MAPE on validation data: {mape}%")
#2000 0.001 12



MAPE on validation data: 20.159053841766266%


In [132]:
np.set_printoptions(precision=4,suppress=True)
print(10 ** y_pred[0:30][:, np.newaxis])
print(10**y_val[0:30].reshape(-1,1))

[[  956.9495]
 [ 2129.8821]
 [  956.9495]
 [  750.3983]
 [12435.294 ]
 [ 1202.5699]
 [ 1985.3902]
 [ 1902.8411]
 [  863.7983]
 [ 3609.3113]
 [ 2333.4883]
 [  809.8306]
 [ 5900.1245]
 [13276.6225]
 [ 1551.3888]
 [  930.1952]
 [  784.1989]
 [ 1961.7974]
 [ 9503.9433]
 [ 2492.5457]
 [ 3589.9071]
 [ 1443.1918]
 [ 3734.8918]
 [  914.4309]
 [ 2175.2253]
 [ 1000.6655]
 [ 2275.5147]
 [  482.2803]
 [ 3454.6676]
 [  687.0136]]
[[ 6817.]
 [ 4905.]
 [ 6887.]
 [ 4174.]
 [16000.]
 [ 1958.]
 [ 2925.]
 [  270.]
 [ 3630.]
 [18000.]
 [15000.]
 [  387.]
 [ 7555.]
 [22000.]
 [    1.]
 [ 1944.]
 [ 1911.]
 [  341.]
 [10000.]
 [ 1506.]
 [ 4283.]
 [  770.]
 [35000.]
 [  235.]
 [  885.]
 [ 1988.]
 [ 1684.]
 [   75.]
 [  202.]
 [  733.]]


### Step 3: Implement Regression

In [200]:
def Regression(dataset, degree):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """

    X = dataset[:, :-1] # All columns except the last one (features)
    y = dataset[:, -1] # Last column (target)

    # Standardize the training data (mean and std are for later use)
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X = (X - mean) / std

    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix

    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    print(f"num_dimensions:{num_dimensions}")
    w = np.zeros(num_dimensions)
    print(f"X_poly.shape:{X_poly.shape}")

    # TODO: Set hyperparameters
    if degree == 2 :
      num_iteration = 30000
      learning_rate = 0.002
    if degree == 1 :
      num_iteration = 500
      learning_rate = 0.01

    # Gradient Descent
    m = len(y)  # Number of data points
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_pred = np.dot(X_poly, w)  # Prediction using current weights
        error = y_pred - y  # Compute the error (residuals)

        # TODO: Compute gradient
        gradient = (1/m) * np.dot(X_poly.T, error)

        # TODO: Update the weights
        w = w - learning_rate * gradient

        # Compute the cost (Mean Squared Error)
        cost = (1/(2*m)) * np.sum(error ** 2)

        # Optionally, print the cost every 100 iterations
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Cost: {cost}")

    return w, mean, std  # Return the weights, mean, and std


### Step 4: Make Prediction

In [201]:

def MakePrediction(w, test_dataset, degree):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    # TODO

    # Create a matrix with polynomial features for the test dataset
    X_poly_test = np.ones((test_dataset.shape[0], 1))  # Start with a column of ones for the intercept term
    for d in range(1, degree + 1):
        X_poly_test = np.hstack((X_poly_test, test_dataset ** d))  # Add x^d terms

    # Compute the predictions using the model's coefficients
    prediction = np.dot(X_poly_test, w)  # dot product of the feature matrix and the weights
    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`



In [ ]:
# TODO
split_ratio = 0.8  # For example, 80% for training and 20% for validation
degree = 1         # Polynomial degree
# (1) Split data
training_data, validation_data = SplitData(training_datalist, split_ratio)

# (2) Preprocess data
training_data = PreprocessData(training_data)
validation_data = PreprocessData(validation_data)

# (3) Train regression model
w, mean, std = Regression(training_data, degree)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
def MAPE(y_true, y_pred):
    """
    Calculates Mean Absolute Percentage Error (MAPE) between the true and predicted values.

    Parameters:
    - y_true (numpy.ndarray): Actual values (ground truth).
    - y_pred (numpy.ndarray): Predicted values.

    Returns:
    - float: MAPE value as a percentage.
    """

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

X_val = validation_data[:, :1]  # All columns except the last one (features)
X_val = (X_val - mean) / std  # Standardize features
y_val = validation_data[:, 1]   # Last column (target)

# Make predictions on validation data
y_val_pred = MakePrediction(w, X_val, degree)

# Calculate MAPE
mape = MAPE(y_val, y_val_pred)
print(f"MAPE on validation data: {mape}%")

# (5) Make prediction of testing dataset and store the values in output_datalist

# Standardize using training mean and std
X_test = (testing_datalist - mean) / std

# Make predictions on the standardized testing data
output_datalist = MakePrediction(w, X_test, degree)


### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [203]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [204]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [205]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [ ]:
# TODO
split_ratio = 0.8  # For example, 80% for training and 20% for validation
degree = 2         # Polynomial degree
# (1) Split data
training_data, validation_data = SplitData(training_datalist, split_ratio)

# (2) Preprocess data
mapping = {'F': 1, 'M': 2}

# Transform gender values
gender_column = training_data[:, 1]  # Extract the gender column
# Apply mapping: map 'F' to 1, 'M' to 2, and leave np.nan unchanged
gender_column_mapped = np.where(gender_column == 'F', 1, np.where(gender_column == 'M', 2, gender_column))
training_data[:, 1] = gender_column_mapped  # Update the dataset with mapped values

# Transform gender values
gender_column = validation_data[:, 1]  # Extract the gender column
# Apply mapping: map 'F' to 1, 'M' to 2, and leave np.nan unchanged
gender_column_mapped = np.where(gender_column == 'F', 1, np.where(gender_column == 'M', 2, gender_column))
validation_data[:, 1] = gender_column_mapped  # Update the dataset with mapped values

training_data = PreprocessData(training_data)
validation_data = PreprocessData(validation_data)

# (3) Train regression model
w, mean, std = Regression(training_data, degree)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
def MAPE(y_true, y_pred):
    """
    Calculates Mean Absolute Percentage Error (MAPE) between the true and predicted values.

    Parameters:
    - y_true (numpy.ndarray): Actual values (ground truth).
    - y_pred (numpy.ndarray): Predicted values.

    Returns:
    - float: MAPE value as a percentage.
    """

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
X_val = validation_data[:, :-1]  # All columns except the last one (features)
X_val = (X_val - mean) / std  # Standardize features
y_val = validation_data[:, -1]   # Last column (target)

# Make predictions on validation data
y_val_pred = MakePrediction(w, X_val, degree)

# Calculate MAPE
mape = MAPE(y_val, y_val_pred)
print(f"MAPE on validation data: {mape}%")

# (5) Make prediction of testing dataset and store the values in output_datalist

# Transform gender values
gender_column = testing_datalist[:, 1]  # Extract the gender column
# Apply mapping: map 'F' to 1, 'M' to 2, and leave np.nan unchanged
gender_column_mapped = np.where(gender_column == 'F', 1, np.where(gender_column == 'M', 2, gender_column))
testing_datalist[:, 1] = gender_column_mapped  # Update the dataset with mapped values

# Standardize using training mean and std
X_test = (testing_datalist - mean) / std

# Make predictions on the standardized testing data
output_datalist = MakePrediction(w, X_test, degree)

# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)